# Imports

In [14]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.document_loaders.csv_loader import CSVLoader
import numpy as np

In [5]:
file_path = ('/Users/saiubc/Desktop/Tranquilo/data_preprocessing/data/dataset.csv')
data = pd.read_csv(file_path)

In [6]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

In [7]:
model_name = "distilbert-base-uncased"  # Choose any model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [8]:
def embedding_function(texts):
    # Tokenize the input texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    # Pass through the model to get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean of the last hidden states as the embedding
    embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return embeddings

In [9]:
dimension = model.config.hidden_size  # Hidden size of the model
index = faiss.IndexFlatL2(dimension)  # L2 distance index
vector_store = FAISS(
    embedding_function=embedding_function,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [16]:
vector_store.add_documents(docs)

RuntimeError: Numpy is not available